<a href="https://colab.research.google.com/github/rimas6/Wathiq/blob/main/SecureSeal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install pillow PyPDF2 -q



In [17]:
import hmac, hashlib
from io import BytesIO
from PIL import Image, PngImagePlugin
from PyPDF2 import PdfReader, PdfWriter

SECRET_KEY = "my_super_secret_key_123"

# ---------- أدوات مساعدة ----------

def normalize_text(text: str) -> str:
    """تطبيع النص قبل الهاش (إزالة الفراغات الزايدة وتوحيدها)."""
    if not text:
        return ""
    # نحول لسترنق ونوحد المسافات
    return " ".join(text.split())

def generate_image_signature(img: Image.Image) -> str:
    """توليد بصمة للصورة بناءً على البكسلات."""
    img_rgb = img.convert("RGB")
    pixel_bytes = img_rgb.tobytes()
    return hmac.new(
        SECRET_KEY.encode(),
        pixel_bytes,
        hashlib.sha256
    ).hexdigest()

def extract_pdf_text(reader: PdfReader) -> str:
    """استخراج كل النص من كل الصفحات."""
    all_text = []
    for page in reader.pages:
        t = page.extract_text() or ""
        all_text.append(t)
    return "\n".join(all_text)

def generate_pdf_signature(pdf_bytes: bytes) -> str:
    """توليد بصمة للـ PDF بناءً على النص داخله."""
    bio = BytesIO(pdf_bytes)
    reader = PdfReader(bio)
    text = extract_pdf_text(reader)
    norm = normalize_text(text)
    return hmac.new(
        SECRET_KEY.encode(),
        norm.encode("utf-8"),
        hashlib.sha256
    ).hexdigest()


In [18]:
def seal_image_bytes(img_bytes: bytes) -> bytes:
    """يأخذ بايتات صورة ويرجع صورة PNG مختومة."""
    img = Image.open(BytesIO(img_bytes))
    signature = generate_image_signature(img)

    meta = PngImagePlugin.PngInfo()
    meta.add_text("secure_seal", signature)

    out_bio = BytesIO()
    img.save(out_bio, format="PNG", pnginfo=meta)
    out_bio.seek(0)
    return out_bio.getvalue()

def seal_pdf_bytes(pdf_bytes: bytes) -> bytes:
    """يضيف الختم داخل Metadata للـ PDF."""
    signature = generate_pdf_signature(pdf_bytes)

    reader = PdfReader(BytesIO(pdf_bytes))
    writer = PdfWriter()

    for page in reader.pages:
        writer.add_page(page)

    # ننسخ الميتاداتا الموجودة + نضيف حقل الختم
    metadata = reader.metadata or {}
    new_metadata = dict(metadata)
    new_metadata["/secure_seal"] = signature
    writer.add_metadata(new_metadata)

    out_bio = BytesIO()
    writer.write(out_bio)
    out_bio.seek(0)
    return out_bio.getvalue()


In [19]:
def verify_image_bytes(img_bytes: bytes) -> str:
    img = Image.open(BytesIO(img_bytes))
    stored_signature = None

    # نحاول نقرأ الختم من text أو info
    if hasattr(img, "text") and "secure_seal" in img.text:
        stored_signature = img.text["secure_seal"]
    if not stored_signature and "secure_seal" in img.info:
        stored_signature = img.info["secure_seal"]

    if not stored_signature:
        return "❌ هذه الصورة غير مختومة."

    recalculated = generate_image_signature(img)

    if hmac.compare_digest(stored_signature, recalculated):
        return "✅ الصورة أصلية ولم يتم التلاعب بها."
    else:
        return "❌ الصورة تم تعديلها أو لا تحتوي على ختم صحيح."


def verify_pdf_bytes(pdf_bytes: bytes) -> str:
    bio = BytesIO(pdf_bytes)
    reader = PdfReader(bio)
    metadata = reader.metadata or {}

    stored_signature = metadata.get("/secure_seal", None)
    if not stored_signature:
        return "❌ هذا المستند غير مختوم."

    recalculated = generate_pdf_signature(pdf_bytes)

    if hmac.compare_digest(stored_signature, recalculated):
        return "✅ المستند أصلي ولم يتم التلاعب به (نصيًا)."
    else:
        return "❌ تم التلاعب بمحتوى المستند أو أن الختم غير صحيح."


In [20]:
from google.colab import files
import os

uploaded = files.upload()  # اختاري صورة أو PDF

for name, data in uploaded.items():
    ext = os.path.splitext(name)[1].lower()

    if ext in [".png", ".jpg", ".jpeg"]:
        sealed = seal_image_bytes(data)
        out_name = f"sealed_{os.path.splitext(name)[0]}.png"
    elif ext == ".pdf":
        sealed = seal_pdf_bytes(data)
        out_name = f"sealed_{name}"
    else:
        print(f"❌ الامتداد {ext} غير مدعوم حالياً.")
        continue

    with open(out_name, "wb") as f:
        f.write(sealed)
    print("✅ تم إنشاء ملف مختوم:", out_name)
    files.download(out_name)


Saving IMG_0703.JPG to IMG_0703.JPG
✅ تم إنشاء ملف مختوم: sealed_IMG_0703.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
from google.colab import files
import os

uploaded = files.upload()  # ارفعي صورة مختومة أو PDF مختوم

for name, data in uploaded.items():
    ext = os.path.splitext(name)[1].lower()

    if ext in [".png", ".jpg", ".jpeg"]:
        result = verify_image_bytes(data)
    elif ext == ".pdf":
        result = verify_pdf_bytes(data)
    else:
        result = f"❌ الامتداد {ext} غير مدعوم."

    print(f"🔍 الملف: {name}")
    print(result)


Saving sealed_IMG_0703.png to sealed_IMG_0703 (1).png
🔍 الملف: sealed_IMG_0703 (1).png
❌ هذه الصورة غير مختومة.
